# Pipeline para Machine Learning

## 1. Importando os pacotes

In [ ]:
import joblib
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

## 2. Carregando e preparando os dados

In [ ]:
# Carregando os dados - dataset
df = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")

# Selecionando os atributos mais relevantes
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
df = df[features + ['Survived']].dropna()  # Removendo as colunas com valores ausentes

# Mostrando os primeiros 5 registros
print(df.head())

### Explicando os atributos utilizados

1. Pclass (Passenger Class)
<br>Tipo: Categórica (mas geralmente tratada como numérica: 1, 2, 3)
<br>Descrição: Classe do bilhete do passageiro.
<br>1 = Primeira classe (mais rica)
<br>2 = Segunda classe
<br>3 = Terceira classe (mais pobre)
<br>Relevância: Passageiros da 1ª classe tiveram mais chance de sobreviver.
<br>
<br>
3. Sex
<br>Tipo: Categórica (male, female)
<br>Descrição: Sexo do passageiro.
<br>Relevância: Mulheres tiveram prioridade nos botes salva-vidas, então o sexo influenciou muito na sobrevivência.
<br><br>
4. Age
<br>Tipo: Numérica (contínua)
<br>Descrição: Idade do passageiro.
<br>Relevância: Crianças pequenas também tiveram prioridade; além disso, idade pode influenciar na capacidade de reagir em uma emergência.
<br><br>
5. SibSp (Siblings/Spouses Aboard)
<br>Tipo: Numérica (inteira)
<br>Descrição: Quantidade de irmãos ou cônjuges a bordo com o passageiro.
<br>SibSp = 0 → Estava sozinho.
<br>SibSp = 1 → Com um irmão(a) ou esposo(a), etc.
<br>Relevância: Estar com familiares pode ter influenciado nas decisões durante o desastre.
<br><br>
7. Parch (Parents/Children Aboard)
<br>Tipo: Numérica (inteira)
<br>Descrição: Número de pais ou filhos a bordo.
<br>Parch = 0 → Sem pais/filhos com ele(a).
<br>Relevância: Assim como SibSp, pode indicar se a pessoa estava em família — o que afeta seu comportamento.
<br><br>
9. Fare
<br>Tipo: Numérica (contínua)
<br>Descrição: Valor da passagem paga pelo passageiro.
<br>Relevância: Em geral, quem pagou mais pode ter tido acesso a melhores cabines (relacionado à Pclass), o que pode impactar na chance de sobreviver.
<br><br>
11. Embarked (Port of Embarkation)
<br>Tipo: Categórica (C, Q, S)
<br>Descrição: Porto de embarque do passageiro:
<br>C = Cherbourg (França)
<br>Q = Queenstown (Irlanda)
<br>S = Southampton (Inglaterra)
<br>Relevância: O porto de embarque pode estar relacionado à classe social ou localização da cabine.

## 3. Definindo as etapas do pré-processamento

In [ ]:
# Definindo os atributos numéricos e categóricos
num_features = ['Age', 'SibSp', 'Parch', 'Fare']
cat_features = ['Pclass', 'Sex', 'Embarked']

# Definindo as normalizações do pré-processamento
num_transformer = StandardScaler()  # Todos os valores numéricos na mesma escala
cat_transformer = OneHotEncoder(handle_unknown='ignore')  # Todos as valores categóricos codificados com o One-hot encoding

# Combinando as normalizações em um objeto de pré-processamento (preprocessor)
preprocessor = ColumnTransformer([
    ('num', num_transformer, num_features),
    ('cat', cat_transformer, cat_features)
])

### 3.1 Exemplos de normalização

🧾 Dados Originais (Antes do Pré-processamento)

| Age | SibSp | Parch | Fare  | Pclass | Sex   | Embarked |
|-----|--------|-------|--------|--------|--------|-----------|
| 22  | 1      | 0     | 7.25   | 3      | male  | S         |
| 38  | 1      | 0     | 71.28  | 1      | female| C         |
| 26  | 0      | 0     | 7.92   | 3      | female| Q         |
| 35  | 1      | 0     | 53.10  | 1      | male  | S         |
| 28  | 0      | 0     | 8.05   | 3      | male  | S         |


⚙️ Dados Após o Pré-processamento

 Numéricos transformados com `StandardScaler`

| Age     | SibSp   | Parch  | Fare     |
|---------|---------|--------|----------|
| -1.20   | 0.87    | 0.00   | -0.82    |
| 1.56    | 0.87    | 0.00   | 1.87     |
| -0.52   | -1.16   | 0.00   | -0.76    |
| 0.96    | 0.87    | 0.00   | 1.17     |
| -0.80   | -1.16   | 0.00   | -0.73    |

 🔹 Categóricos transformados com `OneHotEncoder`

| Pclass_1 | Pclass_3 | Sex_female | Sex_male | Embarked_C | Embarked_Q | Embarked_S |
|----------|----------|------------|----------|------------|------------|-------------|
| 0        | 1        | 0          | 1        | 0          | 0          | 1           |
| 1        | 0        | 1          | 0        | 1          | 0          | 0           |
| 0        | 1        | 1          | 0        | 0          | 1          | 0           |
| 1        | 0        | 0          | 1        | 0          | 0          | 1           |
| 0        | 1        | 0          | 1        | 0          | 0          | 1           |

 ✅ Dados Finais Concatenados

| Age   | SibSp | Parch | Fare  | Pclass_1 | Pclass_3 | Sex_female | Sex_male | Embarked_C | Embarked_Q | Embarked_S |
|-------|--------|--------|--------|------------|------------|-------------|------------|---------------|---------------|---------------|
| -1.20 | 0.87   | 0.00   | -0.82 | 0          | 1          | 0           | 1          | 0             | 0             | 1             |
| 1.56  | 0.87   | 0.00   | 1.87  | 1          | 0          | 1           | 0          | 1             | 0             | 0             |
| -0.52 | -1.16  | 0.00   | -0.76 | 0          | 1          | 1           | 0          | 0             | 1             | 0             |
| 0.96  | 0.87   | 0.00   | 1.17  | 1          | 0          | 0           | 1          | 0             | 0             | 1             |
| -0.80 | -1.16  | 0.00   | -0.73 | 0          | 1          | 0           | 1          | 0             | 0             | 1             |


## 4. Dividindo os dados para treinamento e teste

In [ ]:
# Definindo a varável algo e os atributos
X = df[features]
y = df['Survived']

# Dividindo entre treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Mostrando dimensões de linhas e colunas dos dados
print(f"Dimensões do conjunto de treino - linhas: {X_train.shape[0]} e colunas: {X_train.shape[1]}")
print(f"Dimensões do conjunto de teste - linhas: {X_test.shape[0]} e colunas: {X_test.shape[1]}")

## 5. Criando e treinando o modelo

In [ ]:
# Definindo o pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),  # Transformação dos dados
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))  # Modelo de Machine Learning
])

# Treinando o modelo
pipeline.fit(X_train, y_train)
print("Treinamento do modelo completo!")

## 6. Avaliando o modelo

In [ ]:
# Fazendo as predições
y_pred = pipeline.predict(X_test)

# Verificando a acurácia
accuracy = accuracy_score(y_test, y_pred)
# Proporção de previsões corretas em relação ao total de previsões feitas.
print(f"Acurácia do modelo: {accuracy:.2f} ou  {accuracy* 100:.2f} %") 

## 7. Persistindo e carregando o modelo

In [ ]:
# Persistindo a pipeline com o pré-processamento e o modelo inclusos
joblib.dump(pipeline, 'modelo/modelo_titanic.pkl')

# Carregando a pipeline com o pré-processamento e o modelo inclusos
loaded_pipeline = joblib.load('modelo/modelo_titanic.pkl')

## 8. Fazendo a predição com o modelo carregado

In [ ]:
# Fazendo a predição com a pipeline carregada
sample_data = pd.DataFrame([{'Pclass': 3, 'Sex': 'male', 'Age': 25, 'SibSp': 0, 'Parch': 0, 'Fare': 7.5, 'Embarked': 'S'}])
prediction = loaded_pipeline.predict(sample_data)
print(f"Predição: {'Sobreviveu' if prediction[0] == 1 else 'Não sobreviveu'}")